In [ ]:
from functools import partial
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# tf.config.experimental_run_functions_eagerly(True)

GCS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# EDA

In [ ]:
train_csv = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')

In [ ]:
train_csv

In [ ]:
target_counts = train_csv.target.value_counts()
print(f"0: {target_counts[0]} - {target_counts[0]*100/(target_counts[0]+target_counts[1]):.2f}% of total")
print(f"1: {target_counts[1]} - {target_counts[1]*100/(target_counts[0]+target_counts[1]):.2f}% of total")
print(f"Ratio: {target_counts[0] / target_counts[1]:.2f} : 1")

# Random Over-sampling

In [ ]:
train_df_0 = train_csv[train_csv.target == 0]
train_df_1 = train_csv[train_csv.target == 1]
train_df_1_resampled = train_df_1.sample(target_counts[0], replace=True)
print(f"Upsampled counts - ")
print(f"0: {len(train_df_0)}")
print(f"1: {len(train_df_1_resampled)}")

# Load data from TFRecords

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
train_files = tf.io.gfile.glob(GCS_PATH + '/tfrecords/train*.tfrec')
test_files = tf.io.gfile.glob(GCS_PATH + '/tfrecords/test*.tfrec')

%%time
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMG_SIZE, 3]) # explicit size needed for TPU
    return image

def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_train_vald_dataset(vald_split=0.2, ordered=False):
    dataset = load_dataset(train_files, labeled=True, ordered=ordered)
    n = sum(1 for record in dataset)
    n_vald = int(vald_split * n)
    n_train = n - n_vald
    #train_dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    train_dataset = dataset.take(n_train)
    train_dataset = train_dataset.repeat() # the training dataset must repeat for several epochs
    train_dataset = train_dataset.shuffle(2048)
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    
    vald_dataset = dataset.skip(n_train)
    n1 = sum(1 for rec in vald_dataset)
    vald_dataset = vald_dataset.batch(BATCH_SIZE)
    vald_dataset = vald_dataset.cache()
    vald_dataset = vald_dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    if n_vald != n1:
        print("Validation Dataset sizes - ", n_vald, n1)
    return n_train, train_dataset, n1, vald_dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(test_files, labeled=False, ordered=ordered)
    n = sum(1 for record in dataset)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return n, dataset

n_train, train_dataset, n_vald, vald_dataset = get_train_vald_dataset()
n_test, test_dataset = get_test_dataset(True)
print(f'Dataset: {n_train} training images, {n_vald} validation images, {n_test} unlabeled test images')

# Load data from JPEG

In [ ]:
IMG_SIZE = [1024, 1024]
BATCH_SIZE = 32

In [ ]:
image_dir = "../input/siim-isic-melanoma-classification/jpeg"
train_dir = "train"
test_dir = "test/"

In [ ]:
balanced_train_df = pd.concat([train_df_0, train_df_1_resampled])
balanced_train_df.image_name = balanced_train_df.image_name + ".jpg"
balanced_train_df.target = balanced_train_df.target.astype(str)
balanced_train_df

k = StratifiedKFold(n_splits=1, shuffle=True, random_state=0)
folds = k.split(balanced_train_df, y=balanced_train_df.target)

In [ ]:
train_df, vald_df = train_test_split(balanced_train_df, test_size=0.2, stratify=balanced_train_df.target, shuffle=True, random_state=0)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                 rotation_range=360,
                                 horizontal_flip=True,
                                 vertical_flip=True)
train_dataset = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/siim-isic-melanoma-classification/jpeg/train",
    x_col="image_name",
    y_col="target",
    class_mode="binary",
    batch_size=BATCH_SIZE,
    target_size=IMG_SIZE,
    seed=0)
vald_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
vald_dataset = train_datagen.flow_from_dataframe(
    dataframe=vald_df,
    directory="../input/siim-isic-melanoma-classification/jpeg/train",
    x_col="image_name",
    y_col="target",
    class_mode="binary",
    batch_size=BATCH_SIZE,
    target_size=IMG_SIZE,
    seed=0)

In [ ]:
n_train, n_vald = len(train_dataset), len(vald_dataset)

# Model

In [ ]:
def initialize_model(model_name=""):
    #pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    pretrained_model = tf.keras.applications.Xception(input_shape=[*IMG_SIZE, 3], include_top=False, weights='imagenet')
    #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    # EfficientNet can be loaded through efficientnet.tfkeras library (https://github.com/qubvel/efficientnet)
    #pretrained_model = efficientnet.tfkeras.EfficientNetB0(weights='imagenet', include_top=False)
    
    pretrained_model.trainable = False

    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics=['AUC']
    )

    return model

In [ ]:
with strategy.scope():
    model = initialize_model()

In [ ]:
TRAIN_STEPS = n_train // BATCH_SIZE
VALD_STEPS = n_vald // BATCH_SIZE
EPOCHS = 10
print(f"Training steps = {TRAIN_STEPS}, Validation steps = {VALD_STEPS}")

In [ ]:
t = model.fit(train_dataset, epochs=EPOCHS, steps_per_epoch=TRAIN_STEPS,
                    validation_data=vald_dataset, validation_steps=VALD_STEPS)#, callbacks=[lr_callback])

In [ ]:
plt.plot(t)

In [ ]:
model.save('../working/model.h5')

In [ ]:
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
test_df.image_name = test_df.image_name + ".jpg"

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_dataset = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="../input/siim-isic-melanoma-classification/jpeg/test",
    x_col="image_name",
    y_col=None,
    shuffle=False,
    target_size=IMG_SIZE,
    class_mode=None,
    batch_size=32)

In [ ]:
outs = model.predict(test_dataset)

In [ ]:
pred = pd.DataFrame({'image_name': test_df['image_name'].str.rstrip(".jpg"), 'target': outs.ravel()})
pred

In [ ]:
pred.to_csv('submissions.csv', header=True, index=False)